In [ ]:
! pip install fairlearn
import random
import pandas as pd
import random,time,csv
import numpy as np
import math,copy,os
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sklearn.metrics as metrics
import os
import tempfile
import pandas as pd
import six.moves.urllib as urllib
import pprint
import numpy as np
from pandas import read_csv
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from google.colab import files
from fairlearn.reductions import BoundedGroupLoss, ZeroOneLoss
from sklearn.metrics import mean_absolute_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from fairlearn.metrics import MetricFrame, selection_rate
from sklearn.preprocessing import MinMaxScaler



     |████████████████████████████████| 177 kB 12.4 MB/s 


# Reading the Database

In [ ]:
_DATA_PATH = 'https://storage.googleapis.com/lawschool_dataset/bar_pass_prediction.csv'
# _DATA_FILEPATH = os.path.join(_DATA_ROOT, 'bar_pass_prediction.csv')
 
data = urllib.request.urlopen(_DATA_PATH)
 
_LSAT_DF = pd.read_csv(data)
 
# To simpliy the case study, we will only use the columns that will be used for
# our model.
_COLUMN_NAMES = [
  'dnn_bar_pass_prediction',
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]
 
_LSAT_DF.dropna()
_LSAT_DF['gender'] = _LSAT_DF['gender'].astype(str)
_LSAT_DF['race1'] = _LSAT_DF['race1'].astype(str)
df = _LSAT_DF[_COLUMN_NAMES]

In [ ]:
df.describe()
# temp = list(df.ugpa.values)
#temp.sort()
#print(temp[len(temp)//2])

,dnn_bar_pass_prediction,lsat,pass_bar,ugpa
count,22407.000000,22407.000000,22407.000000,22407.000000
mean,0.980682,36.767907,0.947829,3.215451
std,0.022594,5.463248,0.222377,0.404073
min,0.330740,11.000000,0.000000,1.500000
25%,0.971071,33.000000,1.000000,3.000000
50%,0.983144,37.000000,1.000000,3.200000
75%,0.996526,41.000000,1.000000,3.500000
max,0.999698,48.000000,1.000000,3.900000


In [ ]:
print(df.shape)
df.head()
df['race1'].value_counts()
#df['race1'].value_counts()

(22407, 6)


white    18716
black     1343
hisp      1027
asian      897
other      408
nan         16
Name: race1, dtype: int64

# One Hot Encoding the Categorical Values

In [ ]:
dataset_orig = df
dataset_orig = dataset_orig.dropna()
dataset_orig =  dataset_orig[(dataset_orig.race1 == 'white') | (dataset_orig.race1 == 'black')|(dataset_orig.race1 == 'hisp')|(dataset_orig.race1 == 'asian')]

dataset_orig['gender'] = np.where(dataset_orig['gender'] == 'male', 1, 0)
races = list(dataset_orig['race1'].values)
numRaces = []
for race in races:
  if race == "white":numRaces.append(0)
  elif race == "black" : numRaces.append(1)
  elif race == "hisp" : numRaces.append(2)
  elif race == "asian" : numRaces.append(3)


dataset_orig['race1'] = numRaces
dataset_orig['race1'].value_counts()
# scaler = MinMaxScaler()
# cols = list(dataset_orig.columns)[:-2]
# dataset_orig[cols] = pd.DataFrame(scaler.fit_transform(dataset_orig[cols]),columns = cols)
# dataset_orig=dataset_orig.dropna()


0    18716
1     1343
2     1027
3      897
Name: race1, dtype: int64

In [ ]:
dataset_orig.head()

,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa
0,0.979804,0,44.0,1.0,0,3.5
1,0.979804,0,29.0,1.0,0,3.5
2,0.979804,1,36.0,1.0,0,3.5
3,0.979804,1,39.0,1.0,0,3.5
4,0.979804,1,48.0,1.0,0,3.5


In [ ]:
X, y= dataset_orig.loc[:, dataset_orig.columns != 'ugpa'], dataset_orig['ugpa']



In [ ]:
X.head()

,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1
0,0.979804,0,44.0,1.0,0
1,0.979804,0,29.0,1.0,0
2,0.979804,1,36.0,1.0,0
3,0.979804,1,39.0,1.0,0
4,0.979804,1,48.0,1.0,0


# Clustering the Entry Points using K Mean

In [ ]:
kmean = KMeans(n_clusters = 5, random_state=1)
kmean.fit(X)
KMeans(algorithm='auto', 
       copy_x=True, 
       init='k-means++', # selects initial cluster centers
       max_iter=300,
       n_clusters=5, 
       n_init=10, 
       random_state=1, 
       tol=0.0001, # min. tolerance for distance between clusters
       verbose=0)
clusters = list(kmean.predict(X))
centroids   = kmean.cluster_centers_  
dataset_orig['clusters'] = clusters
X = X.to_numpy()
thresholdDistance = []
for i in range(len(clusters)):
  thresholdDistance.append(np.linalg.norm(X[i]-centroids[clusters[i]]))
dataset_orig['thresholdDistance'] = thresholdDistance

In [ ]:
dataset_orig.head()

,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa,clusters,thresholdDistance
0,0.979804,0,44.0,1.0,0,3.5,2,1.136374
1,0.979804,0,29.0,1.0,0,3.5,3,2.996741
2,0.979804,1,36.0,1.0,0,3.5,4,0.674153
3,0.979804,1,39.0,1.0,0,3.5,0,1.463009
4,0.979804,1,48.0,1.0,0,3.5,2,3.091541


In [ ]:
cluster0 = dataset_orig[dataset_orig['clusters'] == 3]
cluster0['race1'].value_counts()

0    912
1    703
2    267
3     99
Name: race1, dtype: int64

# Fair Class Balancing

In [ ]:
# y = x + rand(0,1)(x-x')
#Smote from synthetic datapoints / orginal points
#Smote generatd from my clan

smoteCluster = pd.DataFrame()
for clusterIndex in range(5):
  cluster0 = dataset_orig[dataset_orig['clusters'] == clusterIndex]
  raceCount = cluster0['race1'].value_counts().to_dict()
  tempRace = [0,0,0,0]
  for indRace in raceCount:tempRace[indRace]=raceCount[indRace]
  raceCount = tempRace
  clusterminimaIndex = tempRace.index(min(tempRace))
  maximaValue = max(raceCount)
  cluster0_3 = cluster0[cluster0['race1'] == clusterminimaIndex]
  cluster0_3 = cluster0_3.drop('clusters', 1)
  cluster0_3 = cluster0_3.drop('race1', 1)
  fCols = list(cluster0_3.columns)
  temp = cluster0_3.to_numpy()
  index = 0
  smoteData = []
  major_minor = maximaValue - raceCount[clusterminimaIndex]
  while len(smoteData) !=major_minor:
    entryPoint = temp[index%raceCount[clusterminimaIndex]]
    src = entryPoint[-1]
    minimaDistance = float("inf")
    minimaIndex = index%raceCount[clusterminimaIndex]
    for i in range(len(temp)):
      if i==index:continue
      dst = temp[i][-1]
      
      dist = abs(src-dst)
      if dst<minimaDistance:
        minimaDistance = dst
        minimaIndex = i
    newPoint = entryPoint + random.uniform(0,1)*(temp[minimaIndex])
    smoteData.append(newPoint)
    index+=1
  smoteData = np.array(smoteData)
  temp = np.concatenate((temp, smoteData), axis=0)
  df = pd.DataFrame(data=temp, columns=fCols)
  rows = df.shape[0]
  df['race1'] = [clusterminimaIndex]*rows
  df['clusters'] = [clusterIndex]*rows
  cluster0 = cluster0[cluster0['race1']!=clusterminimaIndex]
  cluster0 = cluster0.append(df)
  smoteCluster = smoteCluster.append(cluster0)

In [ ]:
for i in range(5):
  print('------Before Fair Class Balancing: Cluster 0----------')
  tempData = dataset_orig[dataset_orig['clusters'] == i]
  print(tempData['race1'].value_counts())
  print('*'*20)
  tempData = smoteCluster[smoteCluster['clusters'] == i]
  print(tempData['race1'].value_counts())
  print('-'*20)  

------Before Fair Class Balancing: Cluster 0----------
0    5232
3     210
2     158
1      88
Name: race1, dtype: int64
********************
1    5232
0    5232
3     210
2     158
Name: race1, dtype: int64
--------------------
------Before Fair Class Balancing: Cluster 0----------
0    4020
1     388
2     320
3     235
Name: race1, dtype: int64
********************
3    4020
0    4020
1     388
2     320
Name: race1, dtype: int64
--------------------
------Before Fair Class Balancing: Cluster 0----------
0    2869
3     132
2      54
1      17
Name: race1, dtype: int64
********************
1    2869
0    2869
3     132
2      54
Name: race1, dtype: int64
--------------------
------Before Fair Class Balancing: Cluster 0----------
0    912
1    703
2    267
3     99
Name: race1, dtype: int64
********************
3    912
0    912
1    703
2    267
Name: race1, dtype: int64
--------------------
------Before Fair Class Balancing: Cluster 0----------
0    5683
2     228
3     221
1     1

In [ ]:
print('Original Dataset : ',dataset_orig.shape)
print('Smote Dataset : ',smoteCluster.shape)

Original Dataset :  (21983, 8)
Smote Dataset :  (40113, 8)


In [ ]:
smoteCluster['race1'].value_counts()

0    18716
1    14875
3     5495
2     1027
Name: race1, dtype: int64

In [ ]:
dataset_orig['race1'].value_counts()

0    18716
1     1343
2     1027
3      897
Name: race1, dtype: int64

In [ ]:
import statistics
ugpa = list(smoteCluster['ugpa'].values)
average = sum(ugpa) / len(ugpa)
median = statistics.median(ugpa)
average_res = []
median_res = []

for entry in ugpa:
  if entry > average:average_res.append(1)
  else:average_res.append(0)
  if entry > median : median_res.append(1)
  else:median_res.append(0)

#smoteCluster['Mean UGPA'] = average_res
#smoteCluster['Median UGPA'] = median_res

3.1


# Saving the dataset into CSV

In [ ]:
smoteCluster.to_csv('Final Fair Class Balancing.csv', sep='\t')
files.download("Final Fair Class Balancing.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/Shareddrives/622 - Privacy/Project Codebase

/content/drive/Shareddrives/622 - Privacy/Project Codebase


In [ ]:
smoteCluster = pd.read_csv('CSV Data/Final Fair Class Balancing.csv',sep = '\t')
smoteCluster.head()

,Unnamed: 0,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa,clusters,thresholdDistance
0,3,0.979804,1.0,39.0,1.0,0,3.5,0,1.463009
1,5,0.979804,0.0,40.0,1.0,0,3.5,0,0.723836
2,7,0.979804,1.0,40.0,1.0,0,3.5,0,0.600363
3,8,0.979804,1.0,40.0,1.0,0,3.5,0,0.600363
4,14,0.979804,1.0,42.0,1.0,0,3.5,0,1.673476


In [ ]:
race_oneHot = pd.get_dummies(smoteCluster.race1, prefix='race')
print(race_oneHot.head())

   race_0  race_1  race_2  race_3
0       1       0       0       0
1       1       0       0       0
2       1       0       0       0
3       1       0       0       0
4       1       0       0       0


In [ ]:
#smoteCluster = smoteCluster.drop('race1',axis = 1)
smoteCluster = smoteCluster.join(race_oneHot)
smoteCluster = smoteCluster.drop('clusters', 1)
smoteCluster.head()

,Unnamed: 0,dnn_bar_pass_prediction,gender,lsat,pass_bar,race1,ugpa,thresholdDistance,race_0,race_1,race_2,race_3
0,3,0.979804,1.0,39.0,1.0,0,3.5,1.463009,1,0,0,0
1,5,0.979804,0.0,40.0,1.0,0,3.5,0.723836,1,0,0,0
2,7,0.979804,1.0,40.0,1.0,0,3.5,0.600363,1,0,0,0
3,8,0.979804,1.0,40.0,1.0,0,3.5,0.600363,1,0,0,0
4,14,0.979804,1.0,42.0,1.0,0,3.5,1.673476,1,0,0,0


In [ ]:
# smoteCluster = smoteCluster.drop('race1', 1)
X = smoteCluster.iloc[:,smoteCluster.columns != 'ugpa']
Y = smoteCluster['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred))      
r2_value = r2_score(Y_test, y_pred)                     

print("Intercept (with race):", model.intercept_)
print('-'*50)
print("Root Mean Square Error (with race):", rmsd)
print('-'*50)
print("R^2 Value (with race):", r2_value)
print('-'*50)

bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(list(X_test['race1'].values))
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print('Bounder Group Loss (with race)  : ',mae_frame.overall)
print('-'*50)
averageCounter = 0
medianCounter = 0
for entry in y_pred:
  if entry > average : averageCounter+=1
  if entry > median : medianCounter+=1
print('Probability of Y=1 (mean) given race : ', averageCounter/len(y_pred))
print('-'*50)
print('Probability of Y=1 (median) given race : ', medianCounter/len(y_pred))
print('-'*50)

Intercept (with race): 0.13081474629061063
--------------------------------------------------
Root Mean Square Error (with race): 0.3582021063080671
--------------------------------------------------
R^2 Value (with race): 0.8720581557197729
--------------------------------------------------
Bounder Group Loss (with race)  :  0.2875403823055883
--------------------------------------------------
Probability of Y=1 (mean) given race :  0.3327932194939549
--------------------------------------------------
Probability of Y=1 (median) given race :  0.38028169014084506
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)


In [ ]:
#smoteCluster = smoteCluster.drop('clusters', 1)
smoteCluster = smoteCluster.drop(['race_1','race_2','race_3'], 1)
X = smoteCluster.iloc[:,smoteCluster.columns != 'ugpa']
Y = smoteCluster['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

X_train = X_train.drop(['race1'],1)
race1 = list(X_test.race1.values)
X_test = X_test.drop(['race1'],1)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred))      
r2_value = r2_score(Y_test, y_pred)                     

print("Intercept (without race):", model.intercept_)
print('-'*50)
print("Root Mean Square Error (without race):", rmsd)
print('-'*50)
print("R^2 Value (without race):", r2_value)
print('-'*50)

bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(race1)
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print('Bounder Group Loss (without race)  : ',mae_frame.overall)
print('-'*50)
averageCounter = 0
medianCounter = 0
for entry in y_pred:
  if entry > average : averageCounter+=1
  if entry > median : medianCounter+=1
print('Probability of Y=1 (mean) without race : ', averageCounter/len(y_pred))
print('-'*50)
print('Probability of Y=1 (median) without race : ', medianCounter/len(y_pred))
print('-'*50)


Intercept (without race): -0.03875241213268721
--------------------------------------------------
Root Mean Square Error (without race): 0.36033923617406205
--------------------------------------------------
R^2 Value (without race): 0.8705269307208227
--------------------------------------------------
Bounder Group Loss (without race)  :  0.2902115996764951
--------------------------------------------------
Probability of Y=1 (mean) without race :  0.33491212763305495
--------------------------------------------------
Probability of Y=1 (median) without race :  0.3805309734513274
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)


In [ ]:
# Vanilla Smote Example
from imblearn.over_sampling import SMOTE
smote = SMOTE()
_DATA_PATH = 'https://storage.googleapis.com/lawschool_dataset/bar_pass_prediction.csv'
data = urllib.request.urlopen(_DATA_PATH)
_LSAT_DF = pd.read_csv(data)
_COLUMN_NAMES = [
  'dnn_bar_pass_prediction',
  'gender',
  'lsat',
  'pass_bar',
  'race1',
  'ugpa',
]
_LSAT_DF.dropna()
_LSAT_DF['gender'] = _LSAT_DF['gender'].astype(str)
_LSAT_DF['race1'] = _LSAT_DF['race1'].astype(str)
df = _LSAT_DF[_COLUMN_NAMES]
dataset_orig = df
dataset_orig = dataset_orig.dropna()
dataset_orig =  dataset_orig[(dataset_orig.race1 == 'white') | (dataset_orig.race1 == 'black')|(dataset_orig.race1 == 'hisp')|(dataset_orig.race1 == 'asian')]

dataset_orig['gender'] = np.where(dataset_orig['gender'] == 'male', 1, 0)
races = list(dataset_orig['race1'].values)
numRaces = []
for race in races:
  if race == "white":numRaces.append(0)
  elif race == "black" : numRaces.append(1)
  elif race == "hisp" : numRaces.append(2)
  elif race == "asian" : numRaces.append(3)


dataset_orig['race1'] = numRaces
dataset_orig['race1'].value_counts()

race_oneHot = pd.get_dummies(smoteCluster.race1, prefix='race')
dataset_orig = dataset_orig.join(race_oneHot)
#dataset_orig = dataset_orig.drop('clusters', 1)

X = dataset_orig.iloc[:,dataset_orig.columns != 'race1']
Y = dataset_orig['race1']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)
X_train_smote,y_train_smote = smote.fit_resample(X_train,Y_train)

dataSmote = pd.DataFrame()
dataSmote = X_train_smote
dataSmote['race1'] = y_train_smote
dataSmote.head()
print(dataset_orig.shape)
print(dataSmote.shape)

import statistics
ugpa = list(dataSmote['ugpa'].values)
average = sum(ugpa) / len(ugpa)
median = statistics.median(ugpa)

X = dataSmote.iloc[:,dataSmote.columns != 'ugpa']
Y = dataSmote['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred))      
r2_value = r2_score(Y_test, y_pred)                     

print("Intercept (with race):", model.intercept_)
print('-'*50)
print("Root Mean Square Error (with race):", rmsd)
print('-'*50)
print("R^2 Value (with race):", r2_value)
print('-'*50)

bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(list(X_test['race1'].values))
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print('Bounder Group Loss (with race)  : ',mae_frame.overall)
print('-'*50)

averageCounter = 0
medianCounter = 0
for entry in y_pred:
  if entry > average : averageCounter+=1
  if entry > median : medianCounter+=1
print('Probability of Y=1 (mean) given race : ', averageCounter/len(y_pred))
print('-'*50)
print('Probability of Y=1 (median) given race : ', medianCounter/len(y_pred))
print('-'*50)

(21983, 10)
(59908, 10)
Intercept (with race): -8.650697658348879
--------------------------------------------------
Root Mean Square Error (with race): 0.2705472627253678
--------------------------------------------------
R^2 Value (with race): 0.5478266467903112
--------------------------------------------------
Bounder Group Loss (with race)  :  0.217756882328121
--------------------------------------------------
Probability of Y=1 (mean) given race :  0.5679352361876148
--------------------------------------------------
Probability of Y=1 (median) given race :  0.5982306793523618
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)


In [ ]:
#smoteCluster = smoteCluster.drop('clusters', 1)
dataSmote = dataSmote.drop(['race_1','race_2','race_3'], 1)
X = dataSmote.iloc[:,dataSmote.columns != 'ugpa']
Y = dataSmote['ugpa']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state= 0)

X_train = X_train.drop(['race1'],1)
race1 = list(X_test.race1.values)
X_test = X_test.drop(['race1'],1)

model = linear_model.LinearRegression()
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

rmsd = np.sqrt(mean_squared_error(Y_test, y_pred))      
r2_value = r2_score(Y_test, y_pred)                     

print("Intercept (without race):", model.intercept_)
print('-'*50)
print("Root Mean Square Error (without race):", rmsd)
print('-'*50)
print("R^2 Value (without race):", r2_value)
print('-'*50)

bgl = BoundedGroupLoss(ZeroOneLoss(), upper_bound=0.1)
sensitive_features = np.array(race1)
mae_frame = MetricFrame(mean_absolute_error,
                        Y_test, y_pred,
                        sensitive_features=pd.Series(sensitive_features, name="SF 0"))
print('Bounder Group Loss (without race)  : ',mae_frame.overall)
print('-'*50)
averageCounter = 0
medianCounter = 0
for entry in y_pred:
  if entry > average : averageCounter+=1
  if entry > median : medianCounter+=1
print('Probability of Y=1 (mean) without race : ', averageCounter/len(y_pred))
print('-'*50)
print('Probability of Y=1 (median) without race : ', medianCounter/len(y_pred))
print('-'*50)


Intercept (without race): -8.778572471965857
--------------------------------------------------
Root Mean Square Error (without race): 0.27353154461748197
--------------------------------------------------
R^2 Value (without race): 0.5377961987954212
--------------------------------------------------
Bounder Group Loss (without race)  :  0.21872165993557416
--------------------------------------------------
Probability of Y=1 (mean) without race :  0.5724419963278251
--------------------------------------------------
Probability of Y=1 (median) without race :  0.6069938240694375
--------------------------------------------------


/usr/local/lib/python3.7/dist-packages/fairlearn/metrics/_metric_frame.py:67: FutureWarning: You have provided 'metrics', 'y_true', 'y_pred' as positional arguments. Please pass them as keyword arguments. From version 0.10.0 passing them as positional arguments will result in an error.
  FutureWarning)
